# L1 Language Models, the Chat Format and Tokens

## Setup
Load the API key and relevant Python libaries.
In this course, we've provided some code that loads the OpenAI API key for you.

In [1]:
# Import required packages
import openai
import os

# Define Azure OpenAI endpoint parameters
openai.api_type = "azure"
openai.api_version = "2023-05-15"
openai.api_base = os.getenv("OPENAI_API_BASE") # Set AOAI endpoint value as env variable
openai.api_key = os.getenv("OPENAI_API_KEY") # Set AOAI API key as env variable
aoai_deployment = os.getenv("OPENAI_API_DEPLOY") # Set AOAI deployment name as env variable

### Helper function
This may look familiar if you took the earlier course "ChatGPT Prompt Engineering for Developers" Course

In [2]:
# Helper function
def get_completion(prompt,
                   # model="gpt-3.5-turbo",
                   engine=aoai_deployment):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        # model=model, # used by original OpenAI endpoint
        engine=engine, # used by Azure OpenAI endpoint
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

### Prompt the model and get a completion

In [3]:
response = get_completion("What is the capital of France?")
print(response)

The capital of France is Paris.


### Tokens

In [4]:
response = get_completion("Take the letters in lollipop \
and reverse them")
print(response)

ppilolol


"lollipop" in reverse should be "popillol"

In [5]:
response = get_completion("""Take the letters in \
l-o-l-l-i-p-o-p and reverse them""")
print(response)

p-o-p-i-l-l-o-l


### Helper function (chat format)
Here's the helper function we'll use in this course.

In [6]:
def get_completion_from_messages(messages,
                                 # model="gpt-3.5-turbo",
                                 engine=aoai_deployment,
                                 temperature=0,
                                 max_tokens=500):
    response = openai.ChatCompletion.create(
        # model=model, # used by original OpenAI endpoint
        engine=engine, # used by Azure OpenAI endpoint
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
        max_tokens=max_tokens, # the maximum number of tokens the model can ouptut 
    )
    return response.choices[0].message["content"]

In [7]:
messages =  [  
{'role':'system', 
 'content':"""You are an assistant who\
 responds in the style of Dr Seuss."""},    
{'role':'user', 
 'content':"""write me a very short poem\
 about a happy carrot"""},  
] 
response = get_completion_from_messages(messages, temperature=1)
print(response)

Oh, how happy is the carrot,
With its bright and brilliant hue.
It's so full of vitamins and joy,
It just makes you smile right through!


In [9]:
# length
messages =  [  
{'role':'system',
 'content':'All your responses must be \
one sentence long.'},    
{'role':'user',
 'content':'write me a story about a happy carrot'},  
] 
response = get_completion_from_messages(messages, temperature=1)
print(response)

Once upon a time, a happy carrot named Carl grew tall and proud in a flourishing garden, surrounded by his fellow vegetables, and he knew he was loved and appreciated by all.


In [10]:
# combined
messages =  [  
{'role':'system',
 'content':"""You are an assistant who \
responds in the style of Dr Seuss. \
All your responses must be one sentence long."""},    
{'role':'user',
 'content':"""write me a story about a happy carrot"""},
] 
response = get_completion_from_messages(messages, 
                                        temperature =1)
print(response)

Once there was a carrot so happy and bright, it sprouted from the ground with so much delight; It danced and it sang, it laughed and it played, and every single day its joy never swayed.


In [11]:
def get_completion_and_token_count(messages,
                                 # model="gpt-3.5-turbo",
                                 engine=aoai_deployment,
                                 temperature=0,
                                 max_tokens=500):
    response = openai.ChatCompletion.create(
        # model=model, # used by original OpenAI endpoint
        engine=engine, # used by Azure OpenAI endpoint
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
        max_tokens=max_tokens, # the maximum number of tokens the model can ouptut 
    )
    content = response.choices[0].message["content"]
    
    token_dict = {
        'prompt_tokens':response['usage']['prompt_tokens'],
        'completion_tokens':response['usage']['completion_tokens'],
        'total_tokens':response['usage']['total_tokens'],
    }

    return content, token_dict

In [12]:
messages = [
    {'role':'system', 
      'content':"""You are an assistant who responds\
      in the style of Dr Seuss."""},
    {'role':'user',
      'content':"""write me a very short poem \
      about a happy carrot"""},  
]
response, token_dict = get_completion_and_token_count(messages)

In [13]:
print(response)

Oh, happy carrot, orange and bright,
You bring such joy with your sweet delight.
Your crunch and flavor, oh so rare,
Make us smile and want to share.
So here's to you, dear happy carrot,
We'll eat you up, without a care!


In [14]:
print(token_dict)

{'prompt_tokens': 39, 'completion_tokens': 55, 'total_tokens': 94}


**Notes on using the OpenAI API outside of this classroom**
To install the OpenAI Python library:

*!pip install openai*

The library needs to be configured with your account's secret key, which is available on the website.

You can either set it as the OPENAI_API_KEY environment variable before using the library:

 *!export OPENAI_API_KEY='sk-...'*

Or, set openai.api_key to its value:

*import openai
openai.api_key = "sk-..."*

**A note about the backslash**
- In the course, we are using a backslash \ to make the text fit on the screen without inserting newline '\n' characters.
- GPT-3 isn't really affected whether you insert newline characters or not. But when working with LLMs in general, you may consider whether newline characters in your prompt may affect the model's performance.